### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import time 

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

### Getting target list 

In [3]:
dataalbums = pd.read_csv('../data/clean/api_albums_clean.csv', encoding='latin1')

In [5]:
dataalbums.head(2)

,albumnamers,albumidspotify,albumnamespotify,artistnamespotify,artistidspotify,releasedate,totaltracks
0,Sgt. Pepper's Lonely Hearts Club Band,1x1jpjDbetGqX0IKCUIBNj,Sgt. Pepper's Lonely Hearts Club Band,Bloco do Sargento Pimenta,3wGWCP3E3tYqj5memYV9Vq,2017-12-08,13.0
1,Pet Sounds,2CNEkSE8TADXRT2AzcEt1b,Pet Sounds,The Beach Boys,3oDbviiivRWhXwIE8hxkVV,1966-05-16,13.0


In [6]:
artist_ids = dataalbums['artistidspotify'].unique().tolist()

In [7]:
artist_ids[0:2]

['3wGWCP3E3tYqj5memYV9Vq', '3oDbviiivRWhXwIE8hxkVV']

### Spotify Connection 

In [8]:
#%pip install pyarrow
#%pip install joblib
#%pip install tqdm
#%pip install spotipy
#%pip install python-dotenv
#%pip install requests 

In [13]:
import requests as rq
import pyarrow
from joblib import Parallel, delayed
from tqdm import tqdm
import pymongo
from passwords import *

In [14]:
# Obtener el TOKEN

AUTH_URL = "https://accounts.spotify.com/api/token"

creds = {
    "grant_type": "client_credentials",
    "client_id": Client_ID,
    "client_secret": Client_secret
}

response = rq.post(AUTH_URL, 
                         data=creds, 
                         headers={"Content-Type": "application/x-www-form-urlencoded"})


TOKEN = response.json()['access_token']

# Conexión con la API

url = "https://api.spotify.com/"

headers = {"Authorization": f'Bearer {TOKEN}'}

response = rq.get(url,headers=headers)
response

<Response [200]>

### Getting Artist Information 

In [15]:
artist_ids = dataalbums['artistidspotify'].unique().tolist()

In [16]:
artist_ids[0:3]

['3wGWCP3E3tYqj5memYV9Vq', '3oDbviiivRWhXwIE8hxkVV', '62F9BiUmjqeXbBztCwiX1U']

In [17]:
artist_urls = [f'https://api.spotify.com/v1/artists/{artist_id}' for artist_id in artist_ids]

artist_url_list = [{'id': artist_id, 'url': url} for artist_id, url in zip(artist_ids, artist_urls)]

In [18]:
# List to store the retrieved artist data
artist_data_list = []

# Function to search and retrieve artist information
def search_artist_info(artist_url):
    time.sleep(0.30)  # Pause between queries to avoid getting banned

    try:
        response = rq.get(artist_url['url'], headers=headers)
        response.raise_for_status()  # Check for request errors
        data = response.json()

        artist_data_list.append({
            'artistidspotify': artist_url['id'],  # Spotify ID for the artist
            'artistnamespotify': data.get('name', None),
            'followers': data.get('followers', {}).get('total', None),
            'popularity': data.get('popularity', None),
            'genres': data.get('genres', None)
            # Add more data points as needed
        })

    except Exception as e:
        print(f"An error occurred: {e}")
        artist_data_list.append({
            'artistidspotify': artist_url['id'],
            'artistnamespotify': None,
            'followers': None,
            'popularity': None,
            'genres': None
            # Add more data points as needed
        })

# Execute the function for each artist URL
for artist_url in artist_url_list:
    search_artist_info(artist_url)

# Create a DataFrame from the list of dictionaries
dataartists = pd.DataFrame(artist_data_list)

In [19]:
dataartists.tail(5)

,artistidspotify,artistnamespotify,followers,popularity,genres
318,05o2ENqv0CV8aD6BWKEaBD,Drayton Farley,60133,52,[modern country pop]
319,0gBvuNzrFCOVaiyKexoYMH,Todrick Hall,402075,52,[strut]
320,6LkVV1P1gvsluy7OOZNG7g,NEAT001,25,0,[]
321,7DEseTqRODmSu3C7jxCHl5,The Boxer Rebellion,109037,42,[indie rock]
322,4pdoRs7yHNXakMobf8M9Oz,SASAMI,61174,33,[]


In [20]:
dataartists.to_csv('../data/clean/api_artists_clean.csv', index=False)